In [15]:
# elm_model_execution.py
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set style untuk visualisasi
plt.style.use('default')
sns.set_palette("husl")

def load_cleaned_data(filename='data_cleaned.pkl'):
    """Load data yang sudah dibersihkan"""
    print("📂 Loading cleaned data...")
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data

def elm_model(X_train, y_train, X_test, n_hidden=50, activation='sigmoid', random_seed=42):
    """ELM model sesuai dokumen Bab 2.2.2 dan 3.3.2"""
    
    # Set random seed untuk reproducibility
    np.random.seed(random_seed)
    
    n_features = X_train.shape[1]
    
    # ✅ SESUAI DOKUMEN: Inisialisasi bobot dengan distribusi normal
    W = np.random.normal(0, 1, (n_features, n_hidden))
    b = np.random.normal(0, 1, n_hidden)

    # Hitung hidden layer output
    H = np.dot(X_train, W) + b

    # Fungsi aktivasi sesuai dokumen
    if activation == 'sigmoid':
        H = 1 / (1 + np.exp(-np.clip(H, -250, 250)))
    elif activation == 'tanh':
        H = np.tanh(H)
    elif activation == 'relu':
        H = np.maximum(0, H)

    # ✅ SESUAI DOKUMEN: Hitung output weights dengan Moore-Penrose pseudoinverse
    I = np.identity(H.shape[1])
    reg_lambda = 1e-8  # ✅ SESUAI DOKUMEN
    
    # Gunakan pseudoinverse untuk numerical stability
    try:
        beta = np.linalg.pinv(H.T @ H + reg_lambda * I) @ H.T @ y_train
    except np.linalg.LinAlgError:
        # Fallback jika matriks singular
        beta = np.linalg.lstsq(H.T @ H + reg_lambda * I, H.T @ y_train, rcond=None)[0]

    # Prediction untuk test set
    H_test = np.dot(X_test, W) + b
    if activation == 'sigmoid':
        H_test = 1 / (1 + np.exp(-np.clip(H_test, -250, 250)))
    elif activation == 'tanh':
        H_test = np.tanh(H_test)
    elif activation == 'relu':
        H_test = np.maximum(0, H_test)

    y_pred = H_test @ beta
    
    # Sigmoid output untuk probabilitas
    y_pred_proba = 1 / (1 + np.exp(-np.clip(y_pred, -250, 250)))
    
    return y_pred_proba

def optimize_random_seed(X, y, preprocessor, best_config, n_seeds=20, n_splits=10):
    """Optimasi random seed untuk meningkatkan stabilitas model"""
    print(f"\n🎲 OPTIMASI RANDOM SEED ({n_seeds} seed, {n_splits}-fold CV)")
    print("=" * 70)
    
    # ✅ GUNAKAN TIMESTAMP untuk memastikan seed berbeda setiap run
    import time
    base_seed = int(time.time() * 1000) % 10000  # Seed dari timestamp
    
    np.random.seed(base_seed)  # Reset random state dengan seed baru
    random_seeds = np.random.randint(0, 10000, n_seeds)
    
    print(f"🔧 Base seed dari timestamp: {base_seed}")
    print(f"📋 Random seeds yang diuji: {random_seeds[:5]}...")  # Tampilkan 5 pertama
    
    seed_results = []
    
    for seed in tqdm(random_seeds, desc="Random Seeds"):
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        fold_scores = []
        
        for train_index, test_index in skf.split(X, y):
            # Split data
            X_train_raw, X_test_raw = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            # Preprocessing di dalam fold
            X_train_processed = preprocessor.fit_transform(X_train_raw)
            X_test_processed = preprocessor.transform(X_test_raw)
            
            # Training ELM dengan seed yang berbeda
            y_pred_proba = elm_model(
                X_train_processed, y_train, X_test_processed,
                n_hidden=best_config['n_hidden'],
                activation=best_config['activation'],
                random_seed=seed
            )
            
            # Hitung F1 score
            fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
            J = tpr - fpr
            best_thresh_idx = np.argmax(J)
            best_threshold = thresholds[best_thresh_idx] if len(thresholds) > 0 else 0.5
            
            y_pred_binary = (y_pred_proba >= best_threshold).astype(int)
            f1 = f1_score(y_test, y_pred_binary, zero_division=0)
            fold_scores.append(f1)
        
        # Hitung statistik untuk seed ini
        mean_f1 = np.mean(fold_scores)
        std_f1 = np.std(fold_scores)
        cv_f1 = (std_f1 / mean_f1) * 100 if mean_f1 > 0 else 100
        
        seed_results.append({
            'seed': seed,
            'mean_f1': mean_f1,
            'std_f1': std_f1,
            'cv_f1': cv_f1,
            'min_f1': np.min(fold_scores),
            'max_f1': np.max(fold_scores)
        })
    
    # Urutkan berdasarkan F1 mean tertinggi dan CV terendah
    seed_results_sorted = sorted(seed_results, 
                                key=lambda x: (x['mean_f1'], -x['cv_f1']), 
                                reverse=True)
    
    # Tampilkan hasil
    print(f"\n📊 HASIL OPTIMASI RANDOM SEED:")
    print("-" * 85)
    print(f"{'Seed':<8} {'Mean F1':<10} {'Std F1':<10} {'CV F1 (%)':<12} {'Min F1':<10} {'Max F1':<10} {'Status':<12}")
    print("-" * 85)
    
    for i, result in enumerate(seed_results_sorted):
        if i == 0:
            status = "✅ TERBAIK"
        elif result['cv_f1'] < 10:
            status = "🟢 STABIL"
        elif result['cv_f1'] < 15:
            status = "🟡 CUKUP"
        else:
            status = "🔴 KURANG"
        
        print(f"{result['seed']:<8} {result['mean_f1']:.4f}    {result['std_f1']:.4f}    {result['cv_f1']:.2f}%        {result['min_f1']:.4f}    {result['max_f1']:.4f}    {status}")
    
    # Pilih seed terbaik (F1 tinggi dan CV rendah)
    best_seed = seed_results_sorted[0]['seed']
    print(f"\n🏆 RANDOM SEED TERBAIK: {best_seed}")
    print(f"   • Mean F1: {seed_results_sorted[0]['mean_f1']:.4f}")
    print(f"   • Std F1: {seed_results_sorted[0]['std_f1']:.4f}")
    print(f"   • CV F1: {seed_results_sorted[0]['cv_f1']:.2f}%")
    
    return best_seed, seed_results_sorted

def optimize_parameters(X, y, preprocessor):
    """Optimasi parameter ELM sesuai Bab 3.3.4"""
    print("\n🎯 OPTIMASI PARAMETER ELM (Bab 3.3.4)")
    print("=" * 60)
    
    hidden_units_list = [50, 100, 150, 200, 250, 300, 400, 500]  # ✅ SESUAI DOKUMEN
    activation_list = ['sigmoid', 'tanh', 'relu']  # ✅ SESUAI DOKUMEN
    
    best_score = 0
    best_params = {}
    results = []
    
    # Quick 5-fold CV untuk screening
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for n_hidden in tqdm(hidden_units_list, desc="Hidden Units"):
        for activation in activation_list:
            fold_scores = []
            
            for train_index, test_index in skf.split(X, y):
                # Split data
                X_train_raw, X_test_raw = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y[train_index], y[test_index]
                
                # Preprocessing di dalam fold
                X_train_processed = preprocessor.fit_transform(X_train_raw)
                X_test_processed = preprocessor.transform(X_test_raw)
                
                # Training ELM
                y_pred_proba = elm_model(X_train_processed, y_train, X_test_processed,
                                       n_hidden=n_hidden, activation=activation, random_seed=42)
                
                # Hitung AUC
                auc_score = roc_auc_score(y_test, y_pred_proba)
                fold_scores.append(auc_score)
            
            mean_score = np.mean(fold_scores)
            results.append({
                'n_hidden': n_hidden,
                'activation': activation,
                'mean_auc': mean_score
            })
            
            if mean_score > best_score:
                best_score = mean_score
                best_params = {
                    'n_hidden': n_hidden,
                    'activation': activation,
                    'mean_auc': mean_score
                }
    
    # Tampilkan hasil optimasi
    print(f"\n🏆 PARAMETER TERBAIK DITEMUKAN:")
    print(f"   • Hidden Units: {best_params['n_hidden']}")
    print(f"   • Activation: {best_params['activation']}")
    print(f"   • AUC Score: {best_params['mean_auc']:.4f}")
    
    return best_params, results

def run_elm_cross_validation(X, y, preprocessor, best_config, n_splits=10):
    """10-Fold Cross Validation sesuai Bab 3.1.4"""
    print(f"\n🚀 10-FOLD CROSS VALIDATION DENGAN KONFIGURASI TERBAIK")
    print("=" * 70)
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Storage untuk hasil
    cv_results = {
        'fold': [], 'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'auc': [],
        'training_time': [], 'best_threshold': [], 'y_true': [], 'y_pred_proba': [],
        'y_pred_binary': [], 'train_size': [], 'test_size': []
    }
    
    print(f"🔬 Konfigurasi ELM: {best_config}")
    
    fold = 1
    for train_index, test_index in tqdm(skf.split(X, y), total=n_splits, desc="10-Fold CV"):
        start_time_fold = time.time()
        
        # Split data
        X_train_raw, X_test_raw = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Preprocessing di dalam fold
        X_train_processed = preprocessor.fit_transform(X_train_raw)
        X_test_processed = preprocessor.transform(X_test_raw)
        
        # Training ELM dengan konfigurasi terbaik
        y_pred_proba = elm_model(
            X_train_processed, y_train, X_test_processed,
            n_hidden=best_config['n_hidden'],
            activation=best_config['activation'],
            random_seed=best_config.get('random_seed', 42)
        )
        
        # Cari threshold optimal
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
        J = tpr - fpr
        best_thresh_idx = np.argmax(J)
        best_threshold = thresholds[best_thresh_idx] if len(thresholds) > 0 else 0.5
        
        # Prediksi dengan threshold optimal
        y_pred_binary = (y_pred_proba >= best_threshold).astype(int)
        
        training_time_fold = time.time() - start_time_fold
        
        # Hitung metrics
        accuracy = accuracy_score(y_test, y_pred_binary)
        precision = precision_score(y_test, y_pred_binary, zero_division=0)
        recall = recall_score(y_test, y_pred_binary, zero_division=0)
        f1 = f1_score(y_test, y_pred_binary, zero_division=0)
        auc_score = roc_auc_score(y_test, y_pred_proba)
        
        # Simpan results
        cv_results['fold'].append(fold)
        cv_results['accuracy'].append(accuracy)
        cv_results['precision'].append(precision)
        cv_results['recall'].append(recall)
        cv_results['f1'].append(f1)
        cv_results['auc'].append(auc_score)
        cv_results['training_time'].append(training_time_fold)
        cv_results['best_threshold'].append(best_threshold)
        cv_results['y_true'].append(y_test)
        cv_results['y_pred_proba'].append(y_pred_proba)
        cv_results['y_pred_binary'].append(y_pred_binary)
        cv_results['train_size'].append(len(X_train_raw))
        cv_results['test_size'].append(len(X_test_raw))
        
        print(f"Fold {fold}: F1={f1:.4f}, AUC={auc_score:.4f}, Time={training_time_fold:.4f}s")
        fold += 1
    
    return cv_results

def analyze_results(cv_results):
    """Analisis hasil cross validation"""
    cv_df = pd.DataFrame({
        'Fold': cv_results['fold'],
        'Accuracy': cv_results['accuracy'],
        'Precision': cv_results['precision'],
        'Recall': cv_results['recall'],
        'F1_Score': cv_results['f1'],
        'AUC': cv_results['auc'],
        'Training_Time': cv_results['training_time'],
        'Best_Threshold': cv_results['best_threshold'],
        'Train_Size': cv_results['train_size'],
        'Test_Size': cv_results['test_size']
    })
    
    # Hitung statistik
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1_Score', 'AUC']
    summary_stats = {}
    
    for metric in metrics:
        values = cv_df[metric]
        summary_stats[metric] = {
            'Mean': np.mean(values),
            'Std': np.std(values),
            'Min': np.min(values),
            'Max': np.max(values),
            'CV': (np.std(values) / np.mean(values)) * 100
        }
    
    print("\n📊 HASIL AKHIR 10-FOLD CROSS VALIDATION:")
    print("-" * 85)
    print(f"{'Metric':<12} {'Mean':<8} {'Std':<8} {'Min':<8} {'Max':<8} {'CV (%)':<8} {'Status':<10}")
    print("-" * 85)
    
    for metric in metrics:
        stats = summary_stats[metric]
        if stats['CV'] < 5:
            status = "SANGAT STABIL"
        elif stats['CV'] < 10:
            status = "STABIL"
        elif stats['CV'] < 15:
            status = "CUKUP STABIL"
        else:
            status = "KURANG STABIL"
        
        print(f"{metric:<12} {stats['Mean']:.4f}   {stats['Std']:.4f}   {stats['Min']:.4f}   {stats['Max']:.4f}   {stats['CV']:.2f}%     {status}")
    
    return cv_df, summary_stats

def main():
    # Load data yang sudah dibersihkan
    data_loaded = load_cleaned_data('data_cleaned.pkl')
    
    data_cleaned = data_loaded['data_cleaned']
    num_cols = data_loaded['num_cols']
    cat_cols = data_loaded['cat_cols']
    preprocessor = data_loaded['preprocessor']
    
    # Prepare X dan y
    X = data_cleaned.drop(columns=['diagnosis_lanjutan'])
    y = data_cleaned['diagnosis_lanjutan'].values
    
    print(f"📊 Data ready: {X.shape}, Class distribution: {pd.Series(y).value_counts().to_dict()}")
    
    # Step 1: Optimasi parameter
    best_params, optimization_results = optimize_parameters(X, y, preprocessor)
    
    # Step 2: Optimasi random seed
    best_seed, seed_results = optimize_random_seed(X, y, preprocessor, best_params, n_seeds=20, n_splits=10)
    
    # Update best_params dengan random seed terbaik
    best_params['random_seed'] = best_seed
    
    # Step 3: Run cross validation dengan parameter terbaik
    cv_results = run_elm_cross_validation(X, y, preprocessor, best_params, n_splits=10)
    
    # Step 4: Analisis hasil
    cv_df, summary_stats = analyze_results(cv_results)
    
    # Step 5: Simpan hasil
    cv_df.to_csv('ELM_Final_Results.csv', index=False)
    
    print(f"\n🎉 ELM MODEL EXECUTION COMPLETED!")
    print(f"   ✅ Parameter optimization: {best_params}")
    print(f"   ✅ Random seed optimization: {best_seed}")
    print(f"   ✅ Cross validation: 10-fold completed")
    print(f"   ✅ Results saved: ELM_Final_Results.csv")
    
    return cv_df, summary_stats, best_params, seed_results

if __name__ == "__main__":
    cv_df, summary_stats, best_params, seed_results = main()

📂 Loading cleaned data...
📊 Data ready: (457, 13), Class distribution: {0: 275, 1: 182}

🎯 OPTIMASI PARAMETER ELM (Bab 3.3.4)


Hidden Units: 100%|██████████| 8/8 [00:05<00:00,  1.59it/s]



🏆 PARAMETER TERBAIK DITEMUKAN:
   • Hidden Units: 150
   • Activation: relu
   • AUC Score: 0.9356

🎲 OPTIMASI RANDOM SEED (20 seed, 10-fold CV)
🔧 Base seed dari timestamp: 6816
📋 Random seeds yang diuji: [5830 4227 3906 7961 4773]...


Random Seeds: 100%|██████████| 20/20 [00:05<00:00,  3.96it/s]



📊 HASIL OPTIMASI RANDOM SEED:
-------------------------------------------------------------------------------------
Seed     Mean F1    Std F1     CV F1 (%)    Min F1     Max F1     Status      
-------------------------------------------------------------------------------------
3315     0.8730    0.0465    5.32%        0.7647    0.9231    ✅ TERBAIK
7961     0.8578    0.0483    5.63%        0.7879    0.9444    🟢 STABIL
7319     0.8519    0.0601    7.06%        0.7568    0.9444    🟢 STABIL
4466     0.8490    0.0393    4.63%        0.7647    0.8947    🟢 STABIL
7970     0.8472    0.0602    7.10%        0.7179    0.9231    🟢 STABIL
6175     0.8428    0.0501    5.95%        0.7647    0.9231    🟢 STABIL
5056     0.8395    0.0580    6.91%        0.7222    0.9189    🟢 STABIL
4227     0.8381    0.0662    7.90%        0.6857    0.9231    🟢 STABIL
9623     0.8381    0.0441    5.26%        0.7660    0.8947    🟢 STABIL
7083     0.8348    0.0411    4.92%        0.7568    0.8947    🟢 STABIL
7522   

10-Fold CV:  40%|████      | 4/10 [00:00<00:00, 36.82it/s]

Fold 1: F1=0.8718, AUC=0.9246, Time=0.0203s
Fold 2: F1=0.9143, AUC=0.9643, Time=0.0193s
Fold 3: F1=0.8387, AUC=0.8631, Time=0.0194s
Fold 4: F1=0.8485, AUC=0.9028, Time=0.0214s
Fold 5: F1=0.9231, AUC=0.9841, Time=0.0207s
Fold 6: F1=0.8718, AUC=0.8869, Time=0.0233s
Fold 7: F1=0.8649, AUC=0.9454, Time=0.0322s


10-Fold CV: 100%|██████████| 10/10 [00:00<00:00, 30.81it/s]

Fold 8: F1=0.9091, AUC=0.9588, Time=0.0325s
Fold 9: F1=0.7647, AUC=0.8436, Time=0.0340s
Fold 10: F1=0.9231, AUC=0.9815, Time=0.0248s

📊 HASIL AKHIR 10-FOLD CROSS VALIDATION:
-------------------------------------------------------------------------------------
Metric       Mean     Std      Min      Max      CV (%)   Status    
-------------------------------------------------------------------------------------
Accuracy     0.9015   0.0332   0.8222   0.9348   3.69%     SANGAT STABIL
Precision    0.8950   0.0669   0.8095   1.0000   7.47%     STABIL
Recall       0.8626   0.0970   0.7222   1.0000   11.24%     CUKUP STABIL
F1_Score     0.8730   0.0465   0.7647   0.9231   5.32%     STABIL
AUC          0.9255   0.0471   0.8436   0.9841   5.09%     STABIL

🎉 ELM MODEL EXECUTION COMPLETED!
   ✅ Parameter optimization: {'n_hidden': 150, 'activation': 'relu', 'mean_auc': np.float64(0.9355910455910456), 'random_seed': np.int32(3315)}
   ✅ Random seed optimization: 3315
   ✅ Cross validation: 10-f

4065     0.8745    0.0478    5.46%        0.8000    0.9444    ✅ TERBAIK
3058     0.8723    0.0368    4.22%        0.8095    0.9189    ✅ TERBAIK
8425     0.8844    0.0372    4.21%        0.8421    0.9474    ✅ TERBAIK
8899     0.8783    0.0340    3.87%        0.8000    0.9143    ✅ TERBAIK
5195     0.8544    0.0242    2.84%        0.8235    0.9000    ✅ TERBAIK
9795     0.8844    0.0461    5.21%        0.7727    0.9444    ✅ TERBAIK

In [81]:
# elm_model_execution.py
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set style untuk visualisasi
plt.style.use('default')
sns.set_palette("husl")

def load_cleaned_data(filename='data_cleaned.pkl'):
    """Load data yang sudah dibersihkan"""
    print("📂 Loading cleaned data...")
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data

def elm_model(X_train, y_train, X_test, n_hidden=50, activation='sigmoid', random_seed=42, reg_lambda=0.01):
    """ELM model sesuai dokumen Bab 2.2.2 dan 3.3.2"""
    
    # Set random seed untuk reproducibility
    np.random.seed(random_seed)
    
    n_features = X_train.shape[1]
    
    # ✅ SESUAI DOKUMEN: Inisialisasi bobot dengan distribusi normal
    W = np.random.normal(0, 1, (n_features, n_hidden))
    b = np.random.normal(0, 1, n_hidden)

    # Hitung hidden layer output
    H = np.dot(X_train, W) + b

    # Fungsi aktivasi sesuai dokumen
    if activation == 'sigmoid':
        H = 1 / (1 + np.exp(-np.clip(H, -250, 250)))
    elif activation == 'tanh':
        H = np.tanh(H)
    elif activation == 'relu':
        H = np.maximum(0, H)

    # ✅ SESUAI DOKUMEN: Hitung output weights dengan Moore-Penrose pseudoinverse
    I = np.identity(H.shape[1])
    # ✅ PARAMETER REGULARISASI: 0.01 sesuai permintaan
    reg_lambda = reg_lambda
    
    # Gunakan pseudoinverse untuk numerical stability
    try:
        beta = np.linalg.pinv(H.T @ H + reg_lambda * I) @ H.T @ y_train
    except np.linalg.LinAlgError:
        # Fallback jika matriks singular
        beta = np.linalg.lstsq(H.T @ H + reg_lambda * I, H.T @ y_train, rcond=None)[0]

    # Prediction untuk test set
    H_test = np.dot(X_test, W) + b
    if activation == 'sigmoid':
        H_test = 1 / (1 + np.exp(-np.clip(H_test, -250, 250)))
    elif activation == 'tanh':
        H_test = np.tanh(H_test)
    elif activation == 'relu':
        H_test = np.maximum(0, H_test)

    y_pred = H_test @ beta
    
    # Sigmoid output untuk probabilitas
    y_pred_proba = 1 / (1 + np.exp(-np.clip(y_pred, -250, 250)))
    
    return y_pred_proba

def optimize_random_seed(X, y, preprocessor, best_config, n_seeds=20, n_splits=10):
    """Optimasi random seed untuk meningkatkan stabilitas model"""
    print(f"\n🎲 OPTIMASI RANDOM SEED ({n_seeds} seed, {n_splits}-fold CV)")
    print("=" * 70)
    
    # ✅ GUNAKAN TIMESTAMP untuk memastikan seed berbeda setiap run
    base_seed = int(time.time() * 1000) % 10000  # Seed dari timestamp
    
    np.random.seed(base_seed)  # Reset random state dengan seed baru
    random_seeds = np.random.randint(0, 10000, n_seeds)
    
    print(f"🔧 Base seed dari timestamp: {base_seed}")
    print(f"📋 Random seeds yang diuji: {random_seeds[:5]}...")  # Tampilkan 5 pertama
    
    seed_results = []
    
    for seed in tqdm(random_seeds, desc="Random Seeds"):
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
        fold_scores = []
        
        for train_index, test_index in skf.split(X, y):
            # Split data
            X_train_raw, X_test_raw = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y[train_index], y[test_index]
            
            # Preprocessing di dalam fold
            X_train_processed = preprocessor.fit_transform(X_train_raw)
            X_test_processed = preprocessor.transform(X_test_raw)
            
            # Training ELM dengan seed yang berbeda
            y_pred_proba = elm_model(
                X_train_processed, y_train, X_test_processed,
                n_hidden=best_config['n_hidden'],
                activation=best_config['activation'],
                random_seed=seed,
                reg_lambda=best_config.get('reg_lambda', 0.01)  # ✅ Gunakan reg_lambda dari config
            )
            
            # Hitung F1 score
            fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
            J = tpr - fpr
            best_thresh_idx = np.argmax(J)
            best_threshold = thresholds[best_thresh_idx] if len(thresholds) > 0 else 0.5
            
            y_pred_binary = (y_pred_proba >= best_threshold).astype(int)
            f1 = f1_score(y_test, y_pred_binary, zero_division=0)
            fold_scores.append(f1)
        
        # Hitung statistik untuk seed ini
        mean_f1 = np.mean(fold_scores)
        std_f1 = np.std(fold_scores)
        cv_f1 = (std_f1 / mean_f1) * 100 if mean_f1 > 0 else 100
        
        seed_results.append({
            'seed': seed,
            'mean_f1': mean_f1,
            'std_f1': std_f1,
            'cv_f1': cv_f1,
            'min_f1': np.min(fold_scores),
            'max_f1': np.max(fold_scores)
        })
    
    # Urutkan berdasarkan F1 mean tertinggi dan CV terendah
    seed_results_sorted = sorted(seed_results, 
                                key=lambda x: (x['mean_f1'], -x['cv_f1']), 
                                reverse=True)
    
    # Tampilkan hasil
    print(f"\n📊 HASIL OPTIMASI RANDOM SEED:")
    print("-" * 85)
    print(f"{'Seed':<8} {'Mean F1':<10} {'Std F1':<10} {'CV F1 (%)':<12} {'Min F1':<10} {'Max F1':<10} {'Status':<12}")
    print("-" * 85)
    
    for i, result in enumerate(seed_results_sorted):
        if i == 0:
            status = "✅ TERBAIK"
        elif result['cv_f1'] < 10:
            status = "🟢 STABIL"
        elif result['cv_f1'] < 15:
            status = "🟡 CUKUP"
        else:
            status = "🔴 KURANG"
        
        print(f"{result['seed']:<8} {result['mean_f1']:.4f}    {result['std_f1']:.4f}    {result['cv_f1']:.2f}%        {result['min_f1']:.4f}    {result['max_f1']:.4f}    {status}")
    
    # Pilih seed terbaik (F1 tinggi dan CV rendah)
    best_seed = seed_results_sorted[0]['seed']
    print(f"\n🏆 RANDOM SEED TERBAIK: {best_seed}")
    print(f"   • Mean F1: {seed_results_sorted[0]['mean_f1']:.4f}")
    print(f"   • Std F1: {seed_results_sorted[0]['std_f1']:.4f}")
    print(f"   • CV F1: {seed_results_sorted[0]['cv_f1']:.2f}%")
    
    return best_seed, seed_results_sorted

def run_elm_cross_validation(X, y, preprocessor, best_config, n_splits=10):
    """10-Fold Cross Validation sesuai Bab 3.1.4"""
    print(f"\n🚀 10-FOLD CROSS VALIDATION DENGAN KONFIGURASI TERBAIK")
    print("=" * 70)
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Storage untuk hasil
    cv_results = {
        'fold': [], 'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'auc': [],
        'training_time': [], 'best_threshold': [], 'y_true': [], 'y_pred_proba': [],
        'y_pred_binary': [], 'train_size': [], 'test_size': []
    }
    
    print(f"🔬 Konfigurasi ELM: {best_config}")
    
    fold = 1
    for train_index, test_index in tqdm(skf.split(X, y), total=n_splits, desc="10-Fold CV"):
        start_time_fold = time.time()
        
        # Split data
        X_train_raw, X_test_raw = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Preprocessing di dalam fold
        X_train_processed = preprocessor.fit_transform(X_train_raw)
        X_test_processed = preprocessor.transform(X_test_raw)
        
        # Training ELM dengan konfigurasi terbaik
        y_pred_proba = elm_model(
            X_train_processed, y_train, X_test_processed,
            n_hidden=best_config['n_hidden'],
            activation=best_config['activation'],
            random_seed=best_config.get('random_seed', 42),
            reg_lambda=best_config.get('reg_lambda', 0.01)  # ✅ Gunakan reg_lambda dari config
        )
        
        # Cari threshold optimal
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
        J = tpr - fpr
        best_thresh_idx = np.argmax(J)
        best_threshold = thresholds[best_thresh_idx] if len(thresholds) > 0 else 0.5
        
        # Prediksi dengan threshold optimal
        y_pred_binary = (y_pred_proba >= best_threshold).astype(int)
        
        training_time_fold = time.time() - start_time_fold
        
        # Hitung metrics
        accuracy = accuracy_score(y_test, y_pred_binary)
        precision = precision_score(y_test, y_pred_binary, zero_division=0)
        recall = recall_score(y_test, y_pred_binary, zero_division=0)
        f1 = f1_score(y_test, y_pred_binary, zero_division=0)
        auc_score = roc_auc_score(y_test, y_pred_proba)
        
        # Simpan results
        cv_results['fold'].append(fold)
        cv_results['accuracy'].append(accuracy)
        cv_results['precision'].append(precision)
        cv_results['recall'].append(recall)
        cv_results['f1'].append(f1)
        cv_results['auc'].append(auc_score)
        cv_results['training_time'].append(training_time_fold)
        cv_results['best_threshold'].append(best_threshold)
        cv_results['y_true'].append(y_test)
        cv_results['y_pred_proba'].append(y_pred_proba)
        cv_results['y_pred_binary'].append(y_pred_binary)
        cv_results['train_size'].append(len(X_train_raw))
        cv_results['test_size'].append(len(X_test_raw))
        
        print(f"Fold {fold}: F1={f1:.4f}, AUC={auc_score:.4f}, Time={training_time_fold:.4f}s")
        fold += 1
    
    return cv_results

def analyze_results(cv_results):
    """Analisis hasil cross validation"""
    cv_df = pd.DataFrame({
        'Fold': cv_results['fold'],
        'Accuracy': cv_results['accuracy'],
        'Precision': cv_results['precision'],
        'Recall': cv_results['recall'],
        'F1_Score': cv_results['f1'],
        'AUC': cv_results['auc'],
        'Training_Time': cv_results['training_time'],
        'Best_Threshold': cv_results['best_threshold'],
        'Train_Size': cv_results['train_size'],
        'Test_Size': cv_results['test_size']
    })
    
    # Hitung statistik
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1_Score', 'AUC']
    summary_stats = {}
    
    for metric in metrics:
        values = cv_df[metric]
        summary_stats[metric] = {
            'Mean': np.mean(values),
            'Std': np.std(values),
            'Min': np.min(values),
            'Max': np.max(values),
            'CV': (np.std(values) / np.mean(values)) * 100
        }
    
    print("\n📊 HASIL AKHIR 10-FOLD CROSS VALIDATION:")
    print("-" * 85)
    print(f"{'Metric':<12} {'Mean':<8} {'Std':<8} {'Min':<8} {'Max':<8} {'CV (%)':<8} {'Status':<10}")
    print("-" * 85)
    
    for metric in metrics:
        stats = summary_stats[metric]
        if stats['CV'] < 5:
            status = "SANGAT STABIL"
        elif stats['CV'] < 10:
            status = "STABIL"
        elif stats['CV'] < 15:
            status = "CUKUP STABIL"
        else:
            status = "KURANG STABIL"
        
        print(f"{metric:<12} {stats['Mean']:.4f}   {stats['Std']:.4f}   {stats['Min']:.4f}   {stats['Max']:.4f}   {stats['CV']:.2f}%     {status}")
    
    return cv_df, summary_stats

def main():
    # Load data yang sudah dibersihkan
    data_loaded = load_cleaned_data('data_cleaned.pkl')
    
    data_cleaned = data_loaded['data_cleaned']
    num_cols = data_loaded['num_cols']
    cat_cols = data_loaded['cat_cols']
    preprocessor = data_loaded['preprocessor']
    
    # Prepare X dan y
    X = data_cleaned.drop(columns=['diagnosis_lanjutan'])
    y = data_cleaned['diagnosis_lanjutan'].values
    
    print(f"📊 Data ready: {X.shape}, Class distribution: {pd.Series(y).value_counts().to_dict()}")
    
    # ✅ LANGGSUNG GUNAKAN PARAMETER YANG DITENTUKAN: 50 hidden units, sigmoid activation, reg_lambda=0.01
    best_params = {
        'n_hidden': 50,
        'activation': 'sigmoid',
        'reg_lambda': 0.01
    }
    
    print(f"🎯 MENGGUNAKAN KONFIGURASI MANUAL:")
    print(f"   • Hidden Units: {best_params['n_hidden']}")
    print(f"   • Activation: {best_params['activation']}")
    print(f"   • Reg Lambda: {best_params['reg_lambda']}")
    
    # Step 2: Optimasi random seed
    best_seed, seed_results = optimize_random_seed(X, y, preprocessor, best_params, n_seeds=20, n_splits=10)
    
    # Update best_params dengan random seed terbaik
    best_params['random_seed'] = best_seed
    
    # Step 3: Run cross validation dengan parameter terbaik
    cv_results = run_elm_cross_validation(X, y, preprocessor, best_params, n_splits=10)
    
    # Step 4: Analisis hasil
    cv_df, summary_stats = analyze_results(cv_results)
    
    # Step 5: Simpan hasil
    cv_df.to_csv('ELM_Final_Results_Manual.csv', index=False)
    
    print(f"\n🎉 ELM MODEL EXECUTION COMPLETED!")
    print(f"   ✅ Manual configuration: {best_params}")
    print(f"   ✅ Random seed optimization: {best_seed}")
    print(f"   ✅ Cross validation: 10-fold completed")
    print(f"   ✅ Results saved: ELM_Final_Results_Manual.csv")
    
    return cv_df, summary_stats, best_params, seed_results

if __name__ == "__main__":
    cv_df, summary_stats, best_params, seed_results = main()

📂 Loading cleaned data...
📊 Data ready: (457, 13), Class distribution: {0: 275, 1: 182}
🎯 MENGGUNAKAN KONFIGURASI MANUAL:
   • Hidden Units: 50
   • Activation: sigmoid
   • Reg Lambda: 0.01

🎲 OPTIMASI RANDOM SEED (20 seed, 10-fold CV)
🔧 Base seed dari timestamp: 3051
📋 Random seeds yang diuji: [4647 8848 7844 3822 1381]...


Random Seeds: 100%|██████████| 20/20 [00:03<00:00,  6.36it/s]



📊 HASIL OPTIMASI RANDOM SEED:
-------------------------------------------------------------------------------------
Seed     Mean F1    Std F1     CV F1 (%)    Min F1     Max F1     Status      
-------------------------------------------------------------------------------------
2673     0.8811    0.0351    3.99%        0.8108    0.9189    ✅ TERBAIK
8848     0.8751    0.0507    5.79%        0.8108    0.9444    🟢 STABIL
6189     0.8677    0.0487    5.61%        0.8000    0.9730    🟢 STABIL
3141     0.8640    0.0423    4.90%        0.8000    0.9189    🟢 STABIL
2273     0.8610    0.0585    6.80%        0.7619    0.9474    🟢 STABIL
1066     0.8569    0.0385    4.49%        0.8000    0.9189    🟢 STABIL
4754     0.8556    0.0329    3.84%        0.8000    0.9000    🟢 STABIL
4206     0.8514    0.0465    5.47%        0.7586    0.9143    🟢 STABIL
6315     0.8459    0.0369    4.36%        0.7879    0.9000    🟢 STABIL
9799     0.8458    0.0614    7.26%        0.7500    0.9474    🟢 STABIL
2456   

10-Fold CV:  50%|█████     | 5/10 [00:00<00:00, 48.48it/s]

Fold 1: F1=0.9189, AUC=0.9722, Time=0.0130s
Fold 2: F1=0.9091, AUC=0.9425, Time=0.0146s
Fold 3: F1=0.8571, AUC=0.8988, Time=0.0141s
Fold 4: F1=0.8108, AUC=0.9226, Time=0.0128s
Fold 5: F1=0.9189, AUC=0.9385, Time=0.0139s
Fold 6: F1=0.8372, AUC=0.8947, Time=0.0142s
Fold 7: F1=0.8837, AUC=0.9610, Time=0.0154s
Fold 8: F1=0.8889, AUC=0.9424, Time=0.0145s
Fold 9: F1=0.9143, AUC=0.9506, Time=0.0129s
Fold 10: F1=0.8718, AUC=0.9300, Time=0.0140s


10-Fold CV: 100%|██████████| 10/10 [00:00<00:00, 49.14it/s]


📊 HASIL AKHIR 10-FOLD CROSS VALIDATION:
-------------------------------------------------------------------------------------
Metric       Mean     Std      Min      Max      CV (%)   Status    
-------------------------------------------------------------------------------------
Accuracy     0.9016   0.0325   0.8478   0.9348   3.60%     SANGAT STABIL
Precision    0.8643   0.0735   0.7500   1.0000   8.51%     STABIL
Recall       0.9058   0.0560   0.8333   1.0000   6.19%     STABIL
F1_Score     0.8811   0.0351   0.8108   0.9189   3.99%     SANGAT STABIL
AUC          0.9353   0.0235   0.8947   0.9722   2.52%     SANGAT STABIL

🎉 ELM MODEL EXECUTION COMPLETED!
   ✅ Manual configuration: {'n_hidden': 50, 'activation': 'sigmoid', 'reg_lambda': 0.01, 'random_seed': np.int32(2673)}
   ✅ Random seed optimization: 2673
   ✅ Cross validation: 10-fold completed
   ✅ Results saved: ELM_Final_Results_Manual.csv


2667     0.8958    0.0343    3.82%        0.8125    0.9268    ✅ TERBAIK
1063     0.8824    0.0279    3.16%        0.8485    0.9231    ✅ TERBAIK
3199     0.9013    0.0480    5.32%        0.8205    0.9714    ✅ TERBAIK
3057     0.8954    0.0323    3.61%        0.8108    0.9231    ✅ TERBAIK


8670     0.8751    0.0235    2.68%        0.8485    0.9189    🟢 STABIL
7248     0.8620    0.0157    1.83%        0.8387    0.8824    🟢 STABIL

In [8]:
# elm_model_execution.py
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set style untuk visualisasi
plt.style.use('default')
sns.set_palette("husl")

def load_cleaned_data(filename='data_cleaned.pkl'):
    """Load data yang sudah dibersihkan"""
    print("📂 Loading cleaned data...")
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data

def elm_model(X_train, y_train, X_test, n_hidden=50, activation='sigmoid', random_seed=None, reg_lambda=0.01):
    """ELM model sesuai dokumen Bab 2.2.2 dan 3.3.2"""
    
    # Set random seed untuk reproducibility
    if random_seed is not None:
        np.random.seed(random_seed)
    
    n_features = X_train.shape[1]
    
    # ✅ SESUAI DOKUMEN: Inisialisasi bobot dengan distribusi normal
    W = np.random.normal(0, 1, (n_features, n_hidden))
    b = np.random.normal(0, 1, n_hidden)

    # Hitung hidden layer output
    H = np.dot(X_train, W) + b

    # Fungsi aktivasi sesuai dokumen
    if activation == 'sigmoid':
        H = 1 / (1 + np.exp(-np.clip(H, -250, 250)))
    elif activation == 'tanh':
        H = np.tanh(H)
    elif activation == 'relu':
        H = np.maximum(0, H)

    # ✅ SESUAI DOKUMEN: Hitung output weights dengan Moore-Penrose pseudoinverse
    I = np.identity(H.shape[1])
    # ✅ PARAMETER REGULARISASI: 0.01 sesuai permintaan
    reg_lambda = reg_lambda
    
    # Gunakan pseudoinverse untuk numerical stability
    try:
        beta = np.linalg.pinv(H.T @ H + reg_lambda * I) @ H.T @ y_train
    except np.linalg.LinAlgError:
        # Fallback jika matriks singular
        beta = np.linalg.lstsq(H.T @ H + reg_lambda * I, H.T @ y_train, rcond=None)[0]

    # Prediction untuk test set
    H_test = np.dot(X_test, W) + b
    if activation == 'sigmoid':
        H_test = 1 / (1 + np.exp(-np.clip(H_test, -250, 250)))
    elif activation == 'tanh':
        H_test = np.tanh(H_test)
    elif activation == 'relu':
        H_test = np.maximum(0, H_test)

    y_pred = H_test @ beta
    
    # Sigmoid output untuk probabilitas
    y_pred_proba = 1 / (1 + np.exp(-np.clip(y_pred, -250, 250)))
    
    return y_pred_proba

def run_elm_cross_validation(X, y, preprocessor, best_config, n_splits=10):
    """10-Fold Cross Validation sesuai Bab 3.1.4"""
    print(f"\n🚀 10-FOLD CROSS VALIDATION DENGAN KONFIGURASI TERBAIK")
    print("=" * 70)
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Storage untuk hasil
    cv_results = {
        'fold': [], 'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'auc': [],
        'training_time': [], 'best_threshold': [], 'y_true': [], 'y_pred_proba': [],
        'y_pred_binary': [], 'train_size': [], 'test_size': []
    }
    
    print(f"🔬 Konfigurasi ELM: {best_config}")
    
    fold = 1
    for train_index, test_index in tqdm(skf.split(X, y), total=n_splits, desc="10-Fold CV"):
        start_time_fold = time.time()
        
        # Split data
        X_train_raw, X_test_raw = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Preprocessing di dalam fold
        X_train_processed = preprocessor.fit_transform(X_train_raw)
        X_test_processed = preprocessor.transform(X_test_raw)
        
        # Training ELM dengan konfigurasi terbaik
        y_pred_proba = elm_model(
            X_train_processed, y_train, X_test_processed,
            n_hidden=best_config['n_hidden'],
            activation=best_config['activation'],
            random_seed=best_config.get('random_seed'),
            reg_lambda=best_config.get('reg_lambda', 0.01)  # ✅ Gunakan reg_lambda dari config
        )
        
        # Cari threshold optimal
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
        J = tpr - fpr
        best_thresh_idx = np.argmax(J)
        best_threshold = thresholds[best_thresh_idx] if len(thresholds) > 0 else 0.5
        
        # Prediksi dengan threshold optimal
        y_pred_binary = (y_pred_proba >= best_threshold).astype(int)
        
        training_time_fold = time.time() - start_time_fold
        
        # Hitung metrics
        accuracy = accuracy_score(y_test, y_pred_binary)
        precision = precision_score(y_test, y_pred_binary, zero_division=0)
        recall = recall_score(y_test, y_pred_binary, zero_division=0)
        f1 = f1_score(y_test, y_pred_binary, zero_division=0)
        auc_score = roc_auc_score(y_test, y_pred_proba)
        
        # Simpan results
        cv_results['fold'].append(fold)
        cv_results['accuracy'].append(accuracy)
        cv_results['precision'].append(precision)
        cv_results['recall'].append(recall)
        cv_results['f1'].append(f1)
        cv_results['auc'].append(auc_score)
        cv_results['training_time'].append(training_time_fold)
        cv_results['best_threshold'].append(best_threshold)
        cv_results['y_true'].append(y_test)
        cv_results['y_pred_proba'].append(y_pred_proba)
        cv_results['y_pred_binary'].append(y_pred_binary)
        cv_results['train_size'].append(len(X_train_raw))
        cv_results['test_size'].append(len(X_test_raw))
        
        print(f"Fold {fold}: F1={f1:.4f}, AUC={auc_score:.4f}, Time={training_time_fold:.4f}s")
        fold += 1
    
    return cv_results

def analyze_results(cv_results):
    """Analisis hasil cross validation"""
    cv_df = pd.DataFrame({
        'Fold': cv_results['fold'],
        'Accuracy': cv_results['accuracy'],
        'Precision': cv_results['precision'],
        'Recall': cv_results['recall'],
        'F1_Score': cv_results['f1'],
        'AUC': cv_results['auc'],
        'Training_Time': cv_results['training_time'],
        'Best_Threshold': cv_results['best_threshold'],
        'Train_Size': cv_results['train_size'],
        'Test_Size': cv_results['test_size']
    })
    
    # Hitung statistik
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1_Score', 'AUC']
    summary_stats = {}
    
    for metric in metrics:
        values = cv_df[metric]
        summary_stats[metric] = {
            'Mean': np.mean(values),
            'Std': np.std(values),
            'Min': np.min(values),
            'Max': np.max(values),
            'CV': (np.std(values) / np.mean(values)) * 100
        }
    
    print("\n📊 HASIL AKHIR 10-FOLD CROSS VALIDATION:")
    print("-" * 85)
    print(f"{'Metric':<12} {'Mean':<8} {'Std':<8} {'Min':<8} {'Max':<8} {'CV (%)':<8} {'Status':<10}")
    print("-" * 85)
    
    for metric in metrics:
        stats = summary_stats[metric]
        if stats['CV'] < 5:
            status = "SANGAT STABIL"
        elif stats['CV'] < 10:
            status = "STABIL"
        elif stats['CV'] < 15:
            status = "CUKUP STABIL"
        else:
            status = "KURANG STABIL"
        
        print(f"{metric:<12} {stats['Mean']:.4f}   {stats['Std']:.4f}   {stats['Min']:.4f}   {stats['Max']:.4f}   {stats['CV']:.2f}%     {status}")
    
    return cv_df, summary_stats

def main():
    # Load data yang sudah dibersihkan
    data_loaded = load_cleaned_data('data_cleaned.pkl')
    
    data_cleaned = data_loaded['data_cleaned']
    num_cols = data_loaded['num_cols']
    cat_cols = data_loaded['cat_cols']
    preprocessor = data_loaded['preprocessor']
    
    # Prepare X dan y
    X = data_cleaned.drop(columns=['diagnosis_lanjutan'])
    y = data_cleaned['diagnosis_lanjutan'].values
    
    print(f"📊 Data ready: {X.shape}, Class distribution: {pd.Series(y).value_counts().to_dict()}")
    
    # ✅ GENERATE RANDOM SEED YANG BERBEDA SETIAP RUN
    current_random_seed = np.random.randint(0, 10000)
    
    # ✅ LANGGSUNG GUNAKAN PARAMETER YANG DITENTUKAN: 50 hidden units, sigmoid activation, reg_lambda=0.01
    # ✅ DENGAN RANDOM SEED YANG BERBEDA SETIAP RUN
    best_params = {
        'n_hidden': 50,
        'activation': 'sigmoid',
        'reg_lambda': 1e-8,
        'random_seed': 42  # ✅ RANDOM SEED YANG BERBEDA SETIAP RUN
    }
    
    print(f"🎯 MENGGUNAKAN KONFIGURASI MANUAL:")
    print(f"   • Hidden Units: {best_params['n_hidden']}")
    print(f"   • Activation: {best_params['activation']}")
    print(f"   • Reg Lambda: {best_params['reg_lambda']}")
    print(f"   • Random Seed: {best_params['random_seed']} (RANDOM - BERBEDA SETIAP RUN)")
    
    # ✅ LANGSUNG RUN CROSS VALIDATION DENGAN RANDOM SEED
    cv_results = run_elm_cross_validation(X, y, preprocessor, best_params, n_splits=10)
    
    # Step 4: Analisis hasil
    cv_df, summary_stats = analyze_results(cv_results)
    
    # Step 5: Simpan hasil dengan timestamp dan random seed
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f'ELM_Results_Seed_{current_random_seed}_{timestamp}.csv'
    cv_df.to_csv(filename, index=False)
    
    print(f"\n🎉 ELM MODEL EXECUTION COMPLETED!")
    print(f"   ✅ Manual configuration: {best_params}")
    print(f"   ✅ Random seed used: {current_random_seed}")
    print(f"   ✅ Cross validation: 10-fold completed")
    print(f"   ✅ Results saved: {filename}")
    
    return cv_df, summary_stats, best_params

if __name__ == "__main__":
    cv_df, summary_stats, best_params = main()

📂 Loading cleaned data...
📊 Data ready: (457, 13), Class distribution: {0: 275, 1: 182}
🎯 MENGGUNAKAN KONFIGURASI MANUAL:
   • Hidden Units: 50
   • Activation: sigmoid
   • Reg Lambda: 1e-08
   • Random Seed: 42 (RANDOM - BERBEDA SETIAP RUN)

🚀 10-FOLD CROSS VALIDATION DENGAN KONFIGURASI TERBAIK
🔬 Konfigurasi ELM: {'n_hidden': 50, 'activation': 'sigmoid', 'reg_lambda': 1e-08, 'random_seed': 42}


10-Fold CV:  40%|████      | 4/10 [00:00<00:00, 34.52it/s]

Fold 1: F1=0.8500, AUC=0.9405, Time=0.0240s
Fold 2: F1=0.8649, AUC=0.9306, Time=0.0189s
Fold 3: F1=0.7778, AUC=0.8631, Time=0.0222s
Fold 4: F1=0.8649, AUC=0.9206, Time=0.0167s
Fold 5: F1=0.8372, AUC=0.9187, Time=0.0144s
Fold 6: F1=0.8500, AUC=0.9298, Time=0.0156s
Fold 7: F1=0.9500, AUC=0.9883, Time=0.0159s


10-Fold CV:  90%|█████████ | 9/10 [00:00<00:00, 40.77it/s]

Fold 8: F1=0.8333, AUC=0.9177, Time=0.0152s
Fold 9: F1=0.7895, AUC=0.8971, Time=0.0159s


10-Fold CV: 100%|██████████| 10/10 [00:00<00:00, 39.34it/s]

Fold 10: F1=0.7568, AUC=0.8745, Time=0.0208s

📊 HASIL AKHIR 10-FOLD CROSS VALIDATION:
-------------------------------------------------------------------------------------
Metric       Mean     Std      Min      Max      CV (%)   Status    
-------------------------------------------------------------------------------------
Accuracy     0.8641   0.0420   0.8000   0.9565   4.86%     SANGAT STABIL
Precision    0.7989   0.0546   0.7200   0.9048   6.84%     STABIL
Recall       0.8839   0.0764   0.7778   1.0000   8.65%     STABIL
F1_Score     0.8374   0.0518   0.7568   0.9500   6.19%     STABIL
AUC          0.9181   0.0333   0.8631   0.9883   3.63%     SANGAT STABIL

🎉 ELM MODEL EXECUTION COMPLETED!
   ✅ Manual configuration: {'n_hidden': 50, 'activation': 'sigmoid', 'reg_lambda': 1e-08, 'random_seed': 42}
   ✅ Random seed used: 5342
   ✅ Cross validation: 10-fold completed
   ✅ Results saved: ELM_Results_Seed_5342_20251202_203846.csv


In [1]:
# elm_model_execution.py
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set style untuk visualisasi
plt.style.use('default')
sns.set_palette("husl")

def load_cleaned_data(filename='data_cleaned.pkl'):
    """Load data yang sudah dibersihkan"""
    print("📂 Loading cleaned data...")
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data

def elm_model(X_train, y_train, X_test, n_hidden=50, activation='sigmoid', random_seed=None, reg_lambda=1e-8):
    """ELM model sesuai dokumen Bab 2.2.2 dan 3.3.2"""
    
    # Set random seed untuk reproducibility
    if random_seed is not None:
        np.random.seed(random_seed)
    
    n_features = X_train.shape[1]
    
    # ✅ SESUAI PROPOSAL: Inisialisasi bobot dengan distribusi normal
    W = np.random.normal(0, 1, (n_features, n_hidden))
    b = np.random.normal(0, 1, n_hidden)

    # Hitung hidden layer output
    H = np.dot(X_train, W) + b

    # Fungsi aktivasi sesuai proposal
    if activation == 'sigmoid':
        H = 1 / (1 + np.exp(-np.clip(H, -250, 250)))
    elif activation == 'tanh':
        H = np.tanh(H)
    elif activation == 'relu':
        H = np.maximum(0, H)

    # ✅ SESUAI PROPOSAL: Hitung output weights dengan Moore-Penrose pseudoinverse
    I = np.identity(H.shape[1])
    # ✅ SESUAI PROPOSAL: reg_lambda = 1e-8
    reg_lambda = reg_lambda
    
    # Gunakan pseudoinverse untuk numerical stability
    try:
        beta = np.linalg.pinv(H.T @ H + reg_lambda * I) @ H.T @ y_train
    except np.linalg.LinAlgError:
        # Fallback jika matriks singular
        beta = np.linalg.lstsq(H.T @ H + reg_lambda * I, H.T @ y_train, rcond=None)[0]

    # Prediction untuk test set
    H_test = np.dot(X_test, W) + b
    if activation == 'sigmoid':
        H_test = 1 / (1 + np.exp(-np.clip(H_test, -250, 250)))
    elif activation == 'tanh':
        H_test = np.tanh(H_test)
    elif activation == 'relu':
        H_test = np.maximum(0, H_test)

    y_pred = H_test @ beta
    
    # Sigmoid output untuk probabilitas
    y_pred_proba = 1 / (1 + np.exp(-np.clip(y_pred, -250, 250)))
    
    return y_pred_proba

def optimize_parameters(X, y, preprocessor):
    """Optimasi parameter ELM sesuai Bab 3.3.4 Proposal"""
    print("\n🎯 OPTIMASI PARAMETER ELM (Sesuai Bab 3.3.4 Proposal)")
    print("=" * 70)
    
    # ✅ SESUAI PROPOSAL: Hidden units [50, 100, 150, 200, 250, 300, 400, 500]
    hidden_units_list = [50, 100, 150, 200, 250, 300, 400, 500]
    # ✅ SESUAI PROPOSAL: Activation function [sigmoid, tanh, relu]
    activation_list = ['sigmoid', 'tanh', 'relu']
    
    best_score = 0
    best_params = {}
    results = []
    
    # Quick 5-fold CV untuk screening sesuai proposal
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for n_hidden in tqdm(hidden_units_list, desc="Optimasi Hidden Units"):
        for activation in activation_list:
            fold_scores = []
            
            for train_index, test_index in skf.split(X, y):
                # Split data
                X_train_raw, X_test_raw = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y[train_index], y[test_index]
                
                # Preprocessing di dalam fold
                X_train_processed = preprocessor.fit_transform(X_train_raw)
                X_test_processed = preprocessor.transform(X_test_raw)
                
                # Training ELM
                y_pred_proba = elm_model(X_train_processed, y_train, X_test_processed,
                                       n_hidden=n_hidden, activation=activation, 
                                       random_seed=42, reg_lambda=1e-8)
                
                # Hitung AUC
                auc_score = roc_auc_score(y_test, y_pred_proba)
                fold_scores.append(auc_score)
            
            mean_score = np.mean(fold_scores)
            results.append({
                'n_hidden': n_hidden,
                'activation': activation,
                'mean_auc': mean_score
            })
            
            if mean_score > best_score:
                best_score = mean_score
                best_params = {
                    'n_hidden': n_hidden,
                    'activation': activation,
                    'mean_auc': mean_score
                }
    
    # Tampilkan hasil optimasi
    print(f"\n🏆 PARAMETER TERBAIK DITEMUKAN:")
    print(f"   • Hidden Units: {best_params['n_hidden']}")
    print(f"   • Activation: {best_params['activation']}")
    print(f"   • AUC Score: {best_params['mean_auc']:.4f}")
    
    return best_params, results

def run_elm_cross_validation(X, y, preprocessor, best_config, n_splits=10):
    """10-Fold Cross Validation sesuai Bab 3.1.4 Proposal"""
    print(f"\n🚀 10-FOLD CROSS VALIDATION DENGAN KONFIGURASI TERBAIK")
    print("=" * 70)
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Storage untuk hasil
    cv_results = {
        'fold': [], 'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'auc': [],
        'training_time': [], 'best_threshold': [], 'y_true': [], 'y_pred_proba': [],
        'y_pred_binary': [], 'train_size': [], 'test_size': []
    }
    
    print(f"🔬 Konfigurasi ELM: {best_config}")
    
    fold = 1
    for train_index, test_index in tqdm(skf.split(X, y), total=n_splits, desc="10-Fold CV"):
        start_time_fold = time.time()
        
        # Split data
        X_train_raw, X_test_raw = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Preprocessing di dalam fold
        X_train_processed = preprocessor.fit_transform(X_train_raw)
        X_test_processed = preprocessor.transform(X_test_raw)
        
        # Training ELM dengan konfigurasi terbaik
        y_pred_proba = elm_model(
            X_train_processed, y_train, X_test_processed,
            n_hidden=best_config['n_hidden'],
            activation=best_config['activation'],
            random_seed=best_config.get('random_seed', 42),
            reg_lambda=1e-8  # ✅ SESUAI PROPOSAL: 1e-8
        )
        
        # Cari threshold optimal
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
        J = tpr - fpr
        best_thresh_idx = np.argmax(J)
        best_threshold = thresholds[best_thresh_idx] if len(thresholds) > 0 else 0.5
        
        # Prediksi dengan threshold optimal
        y_pred_binary = (y_pred_proba >= best_threshold).astype(int)
        
        training_time_fold = time.time() - start_time_fold
        
        # Hitung metrics
        accuracy = accuracy_score(y_test, y_pred_binary)
        precision = precision_score(y_test, y_pred_binary, zero_division=0)
        recall = recall_score(y_test, y_pred_binary, zero_division=0)
        f1 = f1_score(y_test, y_pred_binary, zero_division=0)
        auc_score = roc_auc_score(y_test, y_pred_proba)
        
        # Simpan results
        cv_results['fold'].append(fold)
        cv_results['accuracy'].append(accuracy)
        cv_results['precision'].append(precision)
        cv_results['recall'].append(recall)
        cv_results['f1'].append(f1)
        cv_results['auc'].append(auc_score)
        cv_results['training_time'].append(training_time_fold)
        cv_results['best_threshold'].append(best_threshold)
        cv_results['y_true'].append(y_test)
        cv_results['y_pred_proba'].append(y_pred_proba)
        cv_results['y_pred_binary'].append(y_pred_binary)
        cv_results['train_size'].append(len(X_train_raw))
        cv_results['test_size'].append(len(X_test_raw))
        
        print(f"Fold {fold}: F1={f1:.4f}, AUC={auc_score:.4f}, Time={training_time_fold:.4f}s")
        fold += 1
    
    return cv_results

def analyze_results(cv_results):
    """Analisis hasil cross validation"""
    cv_df = pd.DataFrame({
        'Fold': cv_results['fold'],
        'Accuracy': cv_results['accuracy'],
        'Precision': cv_results['precision'],
        'Recall': cv_results['recall'],
        'F1_Score': cv_results['f1'],
        'AUC': cv_results['auc'],
        'Training_Time': cv_results['training_time'],
        'Best_Threshold': cv_results['best_threshold'],
        'Train_Size': cv_results['train_size'],
        'Test_Size': cv_results['test_size']
    })
    
    # Hitung statistik
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1_Score', 'AUC']
    summary_stats = {}
    
    for metric in metrics:
        values = cv_df[metric]
        summary_stats[metric] = {
            'Mean': np.mean(values),
            'Std': np.std(values),
            'Min': np.min(values),
            'Max': np.max(values),
            'CV': (np.std(values) / np.mean(values)) * 100
        }
    
    print("\n📊 HASIL AKHIR 10-FOLD CROSS VALIDATION:")
    print("-" * 85)
    print(f"{'Metric':<12} {'Mean':<8} {'Std':<8} {'Min':<8} {'Max':<8} {'CV (%)':<8} {'Status':<10}")
    print("-" * 85)
    
    for metric in metrics:
        stats = summary_stats[metric]
        if stats['CV'] < 5:
            status = "SANGAT STABIL"
        elif stats['CV'] < 10:
            status = "STABIL"
        elif stats['CV'] < 15:
            status = "CUKUP STABIL"
        else:
            status = "KURANG STABIL"
        
        print(f"{metric:<12} {stats['Mean']:.4f}   {stats['Std']:.4f}   {stats['Min']:.4f}   {stats['Max']:.4f}   {stats['CV']:.2f}%     {status}")
    
    return cv_df, summary_stats

def main():
    # Load data yang sudah dibersihkan
    data_loaded = load_cleaned_data('data_cleaned.pkl')
    
    data_cleaned = data_loaded['data_cleaned']
    num_cols = data_loaded['num_cols']
    cat_cols = data_loaded['cat_cols']
    preprocessor = data_loaded['preprocessor']
    
    # Prepare X dan y
    X = data_cleaned.drop(columns=['diagnosis_lanjutan'])
    y = data_cleaned['diagnosis_lanjutan'].values
    
    print(f"📊 Data ready: {X.shape}, Class distribution: {pd.Series(y).value_counts().to_dict()}")
    
    # ✅ SESUAI PROPOSAL: Step 1 - Optimasi parameter
    best_params, optimization_results = optimize_parameters(X, y, preprocessor)
    
    # ✅ SESUAI PROPOSAL: Tambahkan random seed tetap untuk reproducibility
    best_params['random_seed'] = 42
    best_params['reg_lambda'] = 1e-8  # ✅ SESUAI PROPOSAL
    
    print(f"\n🎯 MENGGUNAKAN KONFIGURASI OPTIMAL:")
    print(f"   • Hidden Units: {best_params['n_hidden']}")
    print(f"   • Activation: {best_params['activation']}")
    print(f"   • Reg Lambda: {best_params['reg_lambda']}")
    print(f"   • Random Seed: {best_params['random_seed']} (FIXED - SESUAI PROPOSAL)")
    
    # ✅ SESUAI PROPOSAL: Step 2 - 10-Fold Cross Validation dengan parameter terbaik
    cv_results = run_elm_cross_validation(X, y, preprocessor, best_params, n_splits=10)
    
    # Step 3: Analisis hasil
    cv_df, summary_stats = analyze_results(cv_results)
    
    # Step 4: Simpan hasil dengan nama yang sesuai
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f'ELM_Results_Optimal_{best_params["n_hidden"]}_{best_params["activation"]}_{timestamp}.csv'
    cv_df.to_csv(filename, index=False)
    
    print(f"\n🎉 ELM MODEL EXECUTION COMPLETED!")
    print(f"   ✅ Optimal configuration: {best_params}")
    print(f"   ✅ Cross validation: 10-fold completed")
    print(f"   ✅ Final F1 Score: {summary_stats['F1_Score']['Mean']:.4f}")
    print(f"   ✅ Final AUC: {summary_stats['AUC']['Mean']:.4f}")
    print(f"   ✅ Results saved: {filename}")
    
    return cv_df, summary_stats, best_params

if __name__ == "__main__":
    cv_df, summary_stats, best_params = main()

📂 Loading cleaned data...
📊 Data ready: (457, 13), Class distribution: {0: 275, 1: 182}

🎯 OPTIMASI PARAMETER ELM (Sesuai Bab 3.3.4 Proposal)


Optimasi Hidden Units: 100%|██████████| 8/8 [00:04<00:00,  1.63it/s]



🏆 PARAMETER TERBAIK DITEMUKAN:
   • Hidden Units: 50
   • Activation: relu
   • AUC Score: 0.9268

🎯 MENGGUNAKAN KONFIGURASI OPTIMAL:
   • Hidden Units: 50
   • Activation: relu
   • Reg Lambda: 1e-08
   • Random Seed: 42 (FIXED - SESUAI PROPOSAL)

🚀 10-FOLD CROSS VALIDATION DENGAN KONFIGURASI TERBAIK
🔬 Konfigurasi ELM: {'n_hidden': 50, 'activation': 'relu', 'mean_auc': np.float64(0.9267540267540266), 'random_seed': 42, 'reg_lambda': 1e-08}


10-Fold CV:  50%|█████     | 5/10 [00:00<00:00, 43.95it/s]

Fold 1: F1=0.8889, AUC=0.9603, Time=0.0167s
Fold 2: F1=0.8750, AUC=0.9405, Time=0.0164s
Fold 3: F1=0.8125, AUC=0.8750, Time=0.0149s
Fold 4: F1=0.8205, AUC=0.8988, Time=0.0143s
Fold 5: F1=0.8718, AUC=0.9583, Time=0.0144s
Fold 6: F1=0.7692, AUC=0.8713, Time=0.0121s
Fold 7: F1=0.8108, AUC=0.8967, Time=0.0143s
Fold 8: F1=0.8889, AUC=0.9609, Time=0.0139s
Fold 9: F1=0.8387, AUC=0.9012, Time=0.0144s


10-Fold CV: 100%|██████████| 10/10 [00:00<00:00, 47.11it/s]

Fold 10: F1=0.8824, AUC=0.9527, Time=0.0131s

📊 HASIL AKHIR 10-FOLD CROSS VALIDATION:
-------------------------------------------------------------------------------------
Metric       Mean     Std      Min      Max      CV (%)   Status    
-------------------------------------------------------------------------------------
Accuracy     0.8798   0.0350   0.8043   0.9130   3.97%     SANGAT STABIL
Precision    0.8799   0.0852   0.7500   1.0000   9.68%     STABIL
Recall       0.8246   0.0724   0.7222   0.9444   8.78%     STABIL
F1_Score     0.8459   0.0393   0.7692   0.8889   4.65%     SANGAT STABIL
AUC          0.9216   0.0346   0.8713   0.9609   3.75%     SANGAT STABIL

🎉 ELM MODEL EXECUTION COMPLETED!
   ✅ Optimal configuration: {'n_hidden': 50, 'activation': 'relu', 'mean_auc': np.float64(0.9267540267540266), 'random_seed': 42, 'reg_lambda': 1e-08}
   ✅ Cross validation: 10-fold completed
   ✅ Final F1 Score: 0.8459
   ✅ Final AUC: 0.9216
   ✅ Results saved: ELM_Results_Optimal_50_re

In [183]:
# elm_model_execution.py
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve
from tqdm import tqdm
import time
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import pickle
import warnings
warnings.filterwarnings('ignore')

# Set style untuk visualisasi
plt.style.use('default')
sns.set_palette("husl")

def load_cleaned_data(filename='data_cleaned.pkl'):
    """Load data yang sudah dibersihkan"""
    print("📂 Loading cleaned data...")
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    return data

def elm_model(X_train, y_train, X_test, n_hidden=50, activation='sigmoid', random_seed=None, reg_lambda=1e-8):
    """ELM model sesuai dokumen Bab 2.2.2 dan 3.3.2"""
    
    # Set random seed untuk reproducibility
    if random_seed is not None:
        np.random.seed(random_seed)
    
    n_features = X_train.shape[1]
    
    # ✅ SESUAI PROPOSAL: Inisialisasi bobot dengan distribusi normal
    W = np.random.normal(0, 1, (n_features, n_hidden))
    b = np.random.normal(0, 1, n_hidden)

    # Hitung hidden layer output
    H = np.dot(X_train, W) + b

    # Fungsi aktivasi sesuai proposal
    if activation == 'sigmoid':
        H = 1 / (1 + np.exp(-np.clip(H, -250, 250)))
    elif activation == 'tanh':
        H = np.tanh(H)
    elif activation == 'relu':
        H = np.maximum(0, H)

    # ✅ SESUAI PROPOSAL: Hitung output weights dengan Moore-Penrose pseudoinverse
    I = np.identity(H.shape[1])
    # ✅ SESUAI PROPOSAL: reg_lambda = 1e-8
    reg_lambda = reg_lambda
    
    # Gunakan pseudoinverse untuk numerical stability
    try:
        beta = np.linalg.pinv(H.T @ H + reg_lambda * I) @ H.T @ y_train
    except np.linalg.LinAlgError:
        # Fallback jika matriks singular
        beta = np.linalg.lstsq(H.T @ H + reg_lambda * I, H.T @ y_train, rcond=None)[0]

    # Prediction untuk test set
    H_test = np.dot(X_test, W) + b
    if activation == 'sigmoid':
        H_test = 1 / (1 + np.exp(-np.clip(H_test, -250, 250)))
    elif activation == 'tanh':
        H_test = np.tanh(H_test)
    elif activation == 'relu':
        H_test = np.maximum(0, H_test)

    y_pred = H_test @ beta
    
    # Sigmoid output untuk probabilitas
    y_pred_proba = 1 / (1 + np.exp(-np.clip(y_pred, -250, 250)))
    
    return y_pred_proba

def optimize_parameters_for_stability(X, y, preprocessor):
    """Optimasi parameter ELM dengan PRIORITAS STABILITAS"""
    print("\n🎯 OPTIMASI PARAMETER ELM - PRIORITAS STABILITAS")
    print("=" * 70)
    
    # ✅ SESUAI PROPOSAL: Hidden units [50, 100, 150, 200, 250, 300, 400, 500]
    hidden_units_list = [50, 100, 150, 200, 250, 300, 400, 500]
    # ✅ SESUAI PROPOSAL: Activation function [sigmoid, tanh, relu]
    activation_list = ['sigmoid', 'tanh', 'relu']
    
    best_stability_score = float('inf')  # Lower is better
    best_params = {}
    results = []
    
    # Quick 5-fold CV untuk screening sesuai proposal
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for n_hidden in tqdm(hidden_units_list, desc="Optimasi Stabilitas"):
        for activation in activation_list:
            fold_metrics = {
                'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'auc': []
            }
            
            for train_index, test_index in skf.split(X, y):
                # Split data
                X_train_raw, X_test_raw = X.iloc[train_index], X.iloc[test_index]
                y_train, y_test = y[train_index], y[test_index]
                
                # Preprocessing di dalam fold
                X_train_processed = preprocessor.fit_transform(X_train_raw)
                X_test_processed = preprocessor.transform(X_test_raw)
                
                # Training ELM
                y_pred_proba = elm_model(X_train_processed, y_train, X_test_processed,
                                       n_hidden=n_hidden, activation=activation, 
                                       random_seed=42, reg_lambda=1e-8)
                
                # Cari threshold optimal untuk setiap fold
                fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
                J = tpr - fpr
                best_thresh_idx = np.argmax(J)
                best_threshold = thresholds[best_thresh_idx] if len(thresholds) > 0 else 0.5
                
                # Prediksi dengan threshold optimal
                y_pred_binary = (y_pred_proba >= best_threshold).astype(int)
                
                # Hitung semua metrics
                accuracy = accuracy_score(y_test, y_pred_binary)
                precision = precision_score(y_test, y_pred_binary, zero_division=0)
                recall = recall_score(y_test, y_pred_binary, zero_division=0)
                f1 = f1_score(y_test, y_pred_binary, zero_division=0)
                auc_score = roc_auc_score(y_test, y_pred_proba)
                
                fold_metrics['accuracy'].append(accuracy)
                fold_metrics['precision'].append(precision)
                fold_metrics['recall'].append(recall)
                fold_metrics['f1'].append(f1)
                fold_metrics['auc'].append(auc_score)
            
            # Hitung mean dan CV untuk setiap metrik
            metrics_summary = {}
            for metric_name, values in fold_metrics.items():
                mean_val = np.mean(values)
                std_val = np.std(values)
                cv_val = (std_val / mean_val) * 100 if mean_val > 0 else 100
                metrics_summary[metric_name] = {
                    'mean': mean_val,
                    'std': std_val,
                    'cv': cv_val
                }
            
            # ✅ Hitung STABILITY SCORE (CV rata-rata dari semua metrik utama)
            stability_score = np.mean([
                metrics_summary['f1']['cv'],
                metrics_summary['precision']['cv'],
                metrics_summary['recall']['cv'],
                metrics_summary['accuracy']['cv']
            ])
            
            # ✅ Hitung PERFORMANCE SCORE (rata-rata metrik utama)
            performance_score = np.mean([
                metrics_summary['f1']['mean'],
                metrics_summary['precision']['mean'],
                metrics_summary['recall']['mean'],
                metrics_summary['accuracy']['mean']
            ])
            
            # ✅ Hitung COMPOSITE SCORE (prioritas stabilitas)
            # Bobot: 70% stabilitas, 30% performa
            composite_score = (0.7 * (100 - stability_score)) + (0.3 * performance_score * 100)
            
            results.append({
                'n_hidden': n_hidden,
                'activation': activation,
                'stability_score': stability_score,
                'performance_score': performance_score,
                'composite_score': composite_score,
                'f1_mean': metrics_summary['f1']['mean'],
                'f1_cv': metrics_summary['f1']['cv'],
                'precision_mean': metrics_summary['precision']['mean'],
                'precision_cv': metrics_summary['precision']['cv'],
                'recall_mean': metrics_summary['recall']['mean'],
                'recall_cv': metrics_summary['recall']['cv'],
                'accuracy_mean': metrics_summary['accuracy']['mean'],
                'accuracy_cv': metrics_summary['accuracy']['cv'],
                'auc_mean': metrics_summary['auc']['mean'],
                'auc_cv': metrics_summary['auc']['cv']
            })
            
            if stability_score < best_stability_score:
                best_stability_score = stability_score
                best_params = {
                    'n_hidden': n_hidden,
                    'activation': activation,
                    'stability_score': stability_score,
                    'performance_score': performance_score,
                    'composite_score': composite_score,
                    'f1_mean': metrics_summary['f1']['mean'],
                    'f1_cv': metrics_summary['f1']['cv'],
                    'precision_mean': metrics_summary['precision']['mean'],
                    'precision_cv': metrics_summary['precision']['cv'],
                    'recall_mean': metrics_summary['recall']['mean'],
                    'recall_cv': metrics_summary['recall']['cv'],
                    'accuracy_mean': metrics_summary['accuracy']['mean'],
                    'accuracy_cv': metrics_summary['accuracy']['cv']
                }
    
    # Buat DataFrame dari semua hasil
    results_df = pd.DataFrame(results)
    
    # Urutkan berdasarkan STABILITY SCORE (CV terendah)
    results_df = results_df.sort_values('stability_score')
    
    # Tampilkan semua hasil optimasi dengan fokus pada STABILITAS
    print(f"\n📊 HASIL OPTIMASI BERDASARKAN STABILITAS:")
    print("=" * 120)
    print(f"{'Hidden':<8} {'Activation':<10} {'Stability':<12} {'F1_Mean':<8} {'F1_CV':<8} {'Prec_Mean':<8} {'Prec_CV':<8} {'Rec_Mean':<8} {'Rec_CV':<8} {'Acc_Mean':<8} {'Acc_CV':<8} {'Ranking'}")
    print("=" * 120)
    
    for i, (idx, row) in enumerate(results_df.iterrows(), 1):
        # Tentukan ranking berdasarkan stabilitas
        if row['stability_score'] < 5:
            ranking = "🏆 SANGAT STABIL"
        elif row['stability_score'] < 8:
            ranking = "⭐ STABIL"
        elif row['stability_score'] < 12:
            ranking = "👍 CUKUP STABIL"
        elif row['stability_score'] < 15:
            ranking = "⚠️  KURANG STABIL"
        else:
            ranking = "❌ TIDAK STABIL"
        
        print(f"{row['n_hidden']:<8} {row['activation']:<10} {row['stability_score']:<12.2f} {row['f1_mean']:<8.4f} {row['f1_cv']:<8.2f} {row['precision_mean']:<8.4f} {row['precision_cv']:<8.2f} {row['recall_mean']:<8.4f} {row['recall_cv']:<8.2f} {row['accuracy_mean']:<8.4f} {row['accuracy_cv']:<8.2f} {ranking}")
    
    # Analisis mendalam untuk konfigurasi terbaik
    print(f"\n🔍 ANALISIS KONFIGURASI TERSTABIL:")
    print(f"   • Konfigurasi: {best_params['n_hidden']} hidden units, {best_params['activation']} activation")
    print(f"   • Stability Score (CV rata-rata): {best_params['stability_score']:.2f}%")
    print(f"   • Performance Score: {best_params['performance_score']:.4f}")
    print(f"   • F1-Score: {best_params['f1_mean']:.4f} ± {best_params['f1_cv']:.2f}%")
    print(f"   • Precision: {best_params['precision_mean']:.4f} ± {best_params['precision_cv']:.2f}%")
    print(f"   • Recall: {best_params['recall_mean']:.4f} ± {best_params['recall_cv']:.2f}%")
    print(f"   • Accuracy: {best_params['accuracy_mean']:.4f} ± {best_params['accuracy_cv']:.2f}%")
    
    # Analisis berdasarkan activation function
    print(f"\n📈 STABILITAS BERDASARKAN ACTIVATION FUNCTION:")
    activation_stability = results_df.groupby('activation')['stability_score'].agg(['mean', 'min', 'max'])
    for activation, stats in activation_stability.iterrows():
        print(f"   • {activation:>7}: Rata-rata CV = {stats['mean']:.2f}%, Terbaik = {stats['min']:.2f}%")
    
    # Analisis berdasarkan hidden units
    print(f"\n📈 STABILITAS BERDASARKAN HIDDEN UNITS:")
    hidden_stability = results_df.groupby('n_hidden')['stability_score'].agg(['mean', 'min', 'max'])
    # Ambil 3 terstabil
    top_3_stable = hidden_stability.nsmallest(3, 'mean')
    
    print(f"   • 3 Hidden Units Terstabil:")
    for hidden, stats in top_3_stable.iterrows():
        config_data = results_df[results_df['n_hidden'] == hidden].iloc[0]
        print(f"     - {hidden} units: Rata-rata CV = {stats['mean']:.2f}%, F1 = {config_data['f1_mean']:.4f}")
    
    return best_params, results_df

def run_elm_cross_validation(X, y, preprocessor, best_config, n_splits=10):
    """10-Fold Cross Validation dengan konfigurasi terstabil"""
    print(f"\n🚀 10-FOLD CROSS VALIDATION DENGAN KONFIGURASI TERSTABIL")
    print("=" * 70)
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    
    # Storage untuk hasil
    cv_results = {
        'fold': [], 'accuracy': [], 'precision': [], 'recall': [], 'f1': [], 'auc': [],
        'training_time': [], 'best_threshold': [], 'y_true': [], 'y_pred_proba': [],
        'y_pred_binary': [], 'train_size': [], 'test_size': []
    }
    
    print(f"🔬 Konfigurasi ELM: {best_config}")
    
    fold = 1
    for train_index, test_index in tqdm(skf.split(X, y), total=n_splits, desc="10-Fold CV"):
        start_time_fold = time.time()
        
        # Split data
        X_train_raw, X_test_raw = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Preprocessing di dalam fold
        X_train_processed = preprocessor.fit_transform(X_train_raw)
        X_test_processed = preprocessor.transform(X_test_raw)
        
        # Training ELM dengan konfigurasi terstabil
        y_pred_proba = elm_model(
            X_train_processed, y_train, X_test_processed,
            n_hidden=best_config['n_hidden'],
            activation=best_config['activation'],
            random_seed=best_config.get('random_seed', 42),
            reg_lambda=0.001
        )
        
        # Cari threshold optimal
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
        J = tpr - fpr
        best_thresh_idx = np.argmax(J)
        best_threshold = thresholds[best_thresh_idx] if len(thresholds) > 0 else 0.5
        
        # Prediksi dengan threshold optimal
        y_pred_binary = (y_pred_proba >= best_threshold).astype(int)
        
        training_time_fold = time.time() - start_time_fold
        
        # Hitung metrics
        accuracy = accuracy_score(y_test, y_pred_binary)
        precision = precision_score(y_test, y_pred_binary, zero_division=0)
        recall = recall_score(y_test, y_pred_binary, zero_division=0)
        f1 = f1_score(y_test, y_pred_binary, zero_division=0)
        auc_score = roc_auc_score(y_test, y_pred_proba)
        
        # Simpan results
        cv_results['fold'].append(fold)
        cv_results['accuracy'].append(accuracy)
        cv_results['precision'].append(precision)
        cv_results['recall'].append(recall)
        cv_results['f1'].append(f1)
        cv_results['auc'].append(auc_score)
        cv_results['training_time'].append(training_time_fold)
        cv_results['best_threshold'].append(best_threshold)
        cv_results['y_true'].append(y_test)
        cv_results['y_pred_proba'].append(y_pred_proba)
        cv_results['y_pred_binary'].append(y_pred_binary)
        cv_results['train_size'].append(len(X_train_raw))
        cv_results['test_size'].append(len(X_test_raw))
        
        print(f"Fold {fold}: F1={f1:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, Accuracy={accuracy:.4f}, Time={training_time_fold:.4f}s")
        fold += 1
    
    return cv_results

def analyze_results_stability(cv_results):
    """Analisis hasil cross validation dengan fokus pada stabilitas"""
    cv_df = pd.DataFrame({
        'Fold': cv_results['fold'],
        'Accuracy': cv_results['accuracy'],
        'Precision': cv_results['precision'],
        'Recall': cv_results['recall'],
        'F1_Score': cv_results['f1'],
        'AUC': cv_results['auc'],
        'Training_Time': cv_results['training_time'],
        'Best_Threshold': cv_results['best_threshold'],
        'Train_Size': cv_results['train_size'],
        'Test_Size': cv_results['test_size']
    })
    
    # Hitung statistik dengan fokus pada stabilitas
    metrics = ['Accuracy', 'Precision', 'Recall', 'F1_Score', 'AUC']
    summary_stats = {}
    
    for metric in metrics:
        values = cv_df[metric]
        summary_stats[metric] = {
            'Mean': np.mean(values),
            'Std': np.std(values),
            'Min': np.min(values),
            'Max': np.max(values),
            'CV': (np.std(values) / np.mean(values)) * 100,
            'IQR': np.percentile(values, 75) - np.percentile(values, 25),
            'Range': np.max(values) - np.min(values)
        }
    
    # Hitung overall stability score
    stability_score = np.mean([summary_stats[metric]['CV'] for metric in ['Accuracy', 'Precision', 'Recall', 'F1_Score']])
    
    print("\n📊 HASIL 10-FOLD CROSS VALIDATION - ANALISIS STABILITAS:")
    print("=" * 100)
    print(f"{'Metric':<12} {'Mean':<8} {'Std':<8} {'Min':<8} {'Max':<8} {'Range':<8} {'CV (%)':<8} {'IQR':<8} {'Status':<12}")
    print("=" * 100)
    
    for metric in metrics:
        stats = summary_stats[metric]
        
        # Kriteria stabilitas yang lebih ketat
        if stats['CV'] < 3:
            status = "🌟 SANGAT STABIL"
        elif stats['CV'] < 6:
            status = "✅ STABIL"
        elif stats['CV'] < 10:
            status = "⚠️  CUKUP STABIL"
        elif stats['CV'] < 15:
            status = "🔶 KURANG STABIL"
        else:
            status = "❌ TIDAK STABIL"
        
        print(f"{metric:<12} {stats['Mean']:.4f}   {stats['Std']:.4f}   {stats['Min']:.4f}   {stats['Max']:.4f}   {stats['Range']:.4f}   {stats['CV']:.2f}%     {stats['IQR']:.4f}   {status}")
    
    print(f"\n📈 OVERALL STABILITY SCORE: {stability_score:.2f}%")
    
    # Analisis variasi antar fold
    print(f"\n🔍 ANALISIS VARIASI ANTAR FOLD:")
    print(f"   • Rentang F1-Score: {summary_stats['F1_Score']['Min']:.4f} - {summary_stats['F1_Score']['Max']:.4f}")
    print(f"   • Selisih maks-min F1: {summary_stats['F1_Score']['Range']:.4f}")
    print(f"   • Standard Deviation F1: {summary_stats['F1_Score']['Std']:.4f}")
    print(f"   • Coefficient of Variation F1: {summary_stats['F1_Score']['CV']:.2f}%")
    
    # Cek konsistensi performa
    consistent_folds = sum(1 for f1 in cv_results['f1'] if abs(f1 - summary_stats['F1_Score']['Mean']) <= summary_stats['F1_Score']['Std'])
    print(f"   • Jumlah fold dengan performa konsisten: {consistent_folds}/{len(cv_results['f1'])}")
    
    return cv_df, summary_stats, stability_score

def main():
    # Load data yang sudah dibersihkan
    data_loaded = load_cleaned_data('data_cleaned.pkl')
    
    data_cleaned = data_loaded['data_cleaned']
    num_cols = data_loaded['num_cols']
    cat_cols = data_loaded['cat_cols']
    preprocessor = data_loaded['preprocessor']
    
    # Prepare X dan y
    X = data_cleaned.drop(columns=['diagnosis_lanjutan'])
    y = data_cleaned['diagnosis_lanjutan'].values
    
    print(f"📊 Data ready: {X.shape}, Class distribution: {pd.Series(y).value_counts().to_dict()}")
    
    # ✅ OPTIMASI DENGAN PRIORITAS STABILITAS
    best_params, all_results_df = optimize_parameters_for_stability(X, y, preprocessor)
    
    # ✅ Simpan semua hasil optimasi ke CSV
    all_results_df.to_csv('ELM_Optimization_Stability_Results.csv', index=False)
    print(f"\n💾 Semua hasil optimasi stabilitas disimpan: ELM_Optimization_Stability_Results.csv")
    
    # ✅ Tambahkan random seed tetap untuk reproducibility
    best_params['random_seed'] = 42
    best_params['reg_lambda'] = 0.001
    
    print(f"\n🎯 MENGGUNAKAN KONFIGURASI TERSTABIL:")
    print(f"   • Hidden Units: {best_params['n_hidden']}")
    print(f"   • Activation: {best_params['activation']}")
    print(f"   • Reg Lambda: {best_params['reg_lambda']}")
    print(f"   • Stability Score: {best_params['stability_score']:.2f}%")
    
    # ✅ 10-Fold Cross Validation dengan konfigurasi terstabil
    cv_results = run_elm_cross_validation(X, y, preprocessor, best_params, n_splits=10)
    
    # Analisis hasil dengan fokus stabilitas
    cv_df, summary_stats, overall_stability = analyze_results_stability(cv_results)
    
    # Simpan hasil final
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    filename = f'ELM_Stable_Results_{best_params["n_hidden"]}_{best_params["activation"]}_{timestamp}.csv'
    cv_df.to_csv(filename, index=False)
    
    print(f"\n🎉 ELM MODEL EXECUTION COMPLETED - FOKUS STABILITAS!")
    print(f"   ✅ Most stable configuration: {best_params['n_hidden']} {best_params['activation']}")
    print(f"   ✅ Overall Stability Score: {overall_stability:.2f}%")
    print(f"   ✅ Final F1 Score: {summary_stats['F1_Score']['Mean']:.4f} ± {summary_stats['F1_Score']['CV']:.2f}%")
    print(f"   ✅ Final Precision: {summary_stats['Precision']['Mean']:.4f} ± {summary_stats['Precision']['CV']:.2f}%")
    print(f"   ✅ Final Recall: {summary_stats['Recall']['Mean']:.4f} ± {summary_stats['Recall']['CV']:.2f}%")
    print(f"   ✅ Final Accuracy: {summary_stats['Accuracy']['Mean']:.4f} ± {summary_stats['Accuracy']['CV']:.2f}%")
    print(f"   ✅ Results saved: {filename}")
    
    return cv_df, summary_stats, best_params, all_results_df

if __name__ == "__main__":
    cv_df, summary_stats, best_params, all_results_df = main()

📂 Loading cleaned data...
📊 Data ready: (457, 13), Class distribution: {0: 275, 1: 182}

🎯 OPTIMASI PARAMETER ELM - PRIORITAS STABILITAS


Optimasi Stabilitas: 100%|██████████| 8/8 [00:05<00:00,  1.39it/s]



📊 HASIL OPTIMASI BERDASARKAN STABILITAS:
Hidden   Activation Stability    F1_Mean  F1_CV    Prec_Mean Prec_CV  Rec_Mean Rec_CV   Acc_Mean Acc_CV   Ranking
150      relu       4.64         0.8518   4.39     0.8090   5.24     0.9011   5.34     0.8752   3.60     🏆 SANGAT STABIL
200      relu       5.20         0.7792   3.64     0.7502   5.55     0.8180   8.88     0.8162   2.71     ⭐ STABIL
200      sigmoid    5.41         0.7749   4.47     0.7588   6.55     0.7965   7.22     0.8161   3.41     ⭐ STABIL
100      relu       5.44         0.8382   4.95     0.7902   6.53     0.8953   5.99     0.8621   4.28     ⭐ STABIL
100      sigmoid    5.53         0.8028   4.34     0.7682   8.40     0.8461   5.35     0.8337   4.02     ⭐ STABIL
50       sigmoid    5.80         0.8604   4.12     0.8723   6.20     0.8577   10.14    0.8905   2.72     ⭐ STABIL
50       relu       5.89         0.8482   4.13     0.8537   7.12     0.8514   9.27     0.8796   3.05     ⭐ STABIL
150      tanh       7.35         0.7813

10-Fold CV:  30%|███       | 3/10 [00:00<00:00, 28.33it/s]

Fold 1: F1=0.9143, Precision=0.9412, Recall=0.8889, Accuracy=0.9348, Time=0.0295s
Fold 2: F1=0.9444, Precision=0.9444, Recall=0.9444, Accuracy=0.9565, Time=0.0255s
Fold 3: F1=0.8108, Precision=0.7895, Recall=0.8333, Accuracy=0.8478, Time=0.0224s
Fold 4: F1=0.8718, Precision=0.8095, Recall=0.9444, Accuracy=0.8913, Time=0.0227s
Fold 5: F1=0.8947, Precision=0.8500, Recall=0.9444, Accuracy=0.9130, Time=0.0292s
Fold 6: F1=0.8649, Precision=0.8889, Recall=0.8421, Accuracy=0.8913, Time=0.0235s


10-Fold CV:  70%|███████   | 7/10 [00:00<00:00, 30.80it/s]

Fold 7: F1=0.9048, Precision=0.8261, Recall=1.0000, Accuracy=0.9130, Time=0.0222s


10-Fold CV: 100%|██████████| 10/10 [00:00<00:00, 29.73it/s]

Fold 8: F1=0.8372, Precision=0.7200, Recall=1.0000, Accuracy=0.8444, Time=0.0287s
Fold 9: F1=0.8333, Precision=0.8333, Recall=0.8333, Accuracy=0.8667, Time=0.0258s
Fold 10: F1=0.9091, Precision=1.0000, Recall=0.8333, Accuracy=0.9333, Time=0.0243s

📊 HASIL 10-FOLD CROSS VALIDATION - ANALISIS STABILITAS:
Metric       Mean     Std      Min      Max      Range    CV (%)   IQR      Status      
Accuracy     0.8992   0.0359   0.8444   0.9565   0.1121   4.00%     0.0554   ✅ STABIL
Precision    0.8603   0.0795   0.7200   1.0000   0.2800   9.24%     0.1144   ⚠️  CUKUP STABIL
Recall       0.9064   0.0651   0.8333   1.0000   0.1667   7.18%     0.1089   ⚠️  CUKUP STABIL
F1_Score     0.8785   0.0401   0.8108   0.9444   0.1336   4.56%     0.0639   ✅ STABIL
AUC          0.9437   0.0297   0.8948   0.9881   0.0933   3.15%     0.0421   ✅ STABIL

📈 OVERALL STABILITY SCORE: 6.25%

🔍 ANALISIS VARIASI ANTAR FOLD:
   • Rentang F1-Score: 0.8108 - 0.9444
   • Selisih maks-min F1: 0.1336
   • Standard Deviation